In [1]:
!pip install ultralytics


In [3]:
from ultralytics import YOLO
yolo_model = YOLO("yolov8n.pt")


In [ ]:
import tkinter as tk
from tkinter import filedialog, Label, Button
import cv2
from PIL import Image, ImageTk
import numpy as np
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

yolo_model = YOLO("yolov8n.pt") 
color_model = load_model("saved_model/car_color_classifier.h5")
color_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

class_labels = ['beige', 'black', 'blue', 'bronze', 'brown', 'champagne', 'gold', 'gray', 'green', 'maroon', 'orange', 'pink', 'red', 'silver', 'white', 'yellow']
yolo_vehicle_classes = ['car', 'bus', 'truck', 'motorbike']

def detect_and_display(img_path):
    image = cv2.imread(img_path)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = yolo_model.predict(source=rgb_image, conf=0.3, save=False, verbose=False)[0]

    person_count = 0

    for result in results.boxes:
        x1, y1, x2, y2 = map(int, result.xyxy[0])
        class_id = int(result.cls[0])
        label = yolo_model.model.names[class_id]

        if label == 'person':
            person_count += 1
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 255), 2)
        elif label in yolo_vehicle_classes:
            roi = rgb_image[y1:y2, x1:x2]
            if roi.size == 0:
                continue
            roi_resized = cv2.resize(roi, (224, 224))
            roi_preprocessed = img_to_array(roi_resized) / 255.0
            roi_preprocessed = np.expand_dims(roi_preprocessed, axis=0)

            prediction = color_model.predict(roi_preprocessed, verbose=0)
            class_index = np.argmax(prediction[0])
            color = class_labels[class_index]

            box_color = (0, 0, 255) if color == 'blue' else (255, 0, 0)
            label_text = f"{color}"

            cv2.rectangle(image, (x1, y1), (x2, y2), box_color, 2)
            cv2.putText(image, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, box_color, 2)

    cv2.putText(image, f"People: {person_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def upload_image():
    file_path = filedialog.askopenfilename()
    if not file_path:
        return
    processed = detect_and_display(file_path)
    img = Image.fromarray(processed)
    img = img.resize((800, 600))
    img_tk = ImageTk.PhotoImage(img)
    panel.config(image=img_tk)
    panel.image = img_tk

root = tk.Tk()
root.title("Car Color & People Detection - YOLO Edition")
root.geometry("820x680")

panel = Label(root)
panel.pack(pady=10)

btn = Button(root, text="Upload Image", command=upload_image, font=("Arial", 14), bg="lightblue")
btn.pack()

root.mainloop()
